In [59]:
import pandas as pd
import numpy as np
import os
import glob
import geopandas as gpd

In [60]:
root_path = "D:/GeoData/"
Main_CRS = "EPSG:27700"

Import the data into a standard structrue for the geopandas dataframes:

Type | SubType | Name/ID | Easting | Northing

In [148]:
def Import_Points(infile, inputs):
    #Get the before and after columns names
    usecols = list(filter(None,list(inputs.values())[1:5]))
    col_names = list({key: value for key, value in inputs.items() if value in usecols}.keys())

    #Get the data types
    dtype_out = {key: value for key, value in dtypes.items() if key in col_names}
    print(dtype)

    #Import the data
    df = pd.read_csv(infile, usecols=usecols, encoding = "ISO-8859-1", low_memory=False) 

    #Reformat ready for next step
    df.columns = col_names
    df = df.astype(dtype_out)

    df["Type"] = inputs["Type"]

    return df

In [149]:
#source data
rail_infile = root_path + "NaPTANcsv/RailReferences.csv"
rail_inputs = {"Type":"RailwayStations", "SubType":"", "Name":"StationName", "Easting":"Easting", "Northing":"Northing"}

ferry_infile = root_path + "NaPTANcsv/FerryReferences.csv"
ferry_inputs = {"Type":"FerryTerminals", "SubType":"", "Name":"Name", "Easting":"Easting", "Northing":"Northing"}

bus_infile = root_path + "NaPTANcsv/Stops.csv"
bus_inputs = {"Type":"BusStops", "SubType":"", "Name":"CommonName", "Easting":"Easting", "Northing":"Northing"}

acc2020_infile = root_path + "RoadSafety/dft-road-casualty-statistics-accident-2020.csv"
acc2020_inputs = {"Type":"RoadAccidents", "SubType":"", "Name":"accident_reference", "Easting":"location_easting_osgr", "Northing":"location_northing_osgr"}

NSPL_infile = root_path + "NSPL_NOV_2020_UK/Data/NSPL_NOV_2020_UK.csv"
NSPL_inputs = {"Type":"NSPL", "SubType":"", "Name":"pcd", "Easting":"oseast1m", "Northing":"osnrth1m"}

LandReg20_infile = root_path + "LandReg/pp-2020.csv"
LandReg20 = {"Type":"NSPL", "SubType":"", "Name":"pcd", "Easting":"oseast1m", "Northing":"osnrth1m"}

point_pairs = [[rail_infile, rail_inputs],
[ferry_infile, ferry_inputs],
[bus_infile, bus_inputs],
[acc2020_infile, acc2020_inputs],
[NSPL_infile,NSPL_inputs]]


In [150]:
#initiate the standard structure
Points_of_Interest = pd.DataFrame({"Type":"", "SubType":"", "Name":"", "Easting":0, "Northing":0}, index=[0])
dtypes = {"Type":"str", "SubType":"str", "Name":"str", "Easting":np.float64, "Northing":np.float64}
Points_of_Interest = Points_of_Interest.astype(dtypes)
Points_of_Interest = Points_of_Interest.drop(0)

In [151]:
for l in point_pairs:
    Points_of_Interest = Points_of_Interest.append(Import_Points(l[0], l[1]), ignore_index=True)


Points_of_Interest


{'Name': 'str', 'Easting': <class 'numpy.float64'>, 'Northing': <class 'numpy.float64'>}
{'Name': 'str', 'Easting': <class 'numpy.float64'>, 'Northing': <class 'numpy.float64'>}
{'Name': 'str', 'Easting': <class 'numpy.float64'>, 'Northing': <class 'numpy.float64'>}
{'Name': 'str', 'Easting': <class 'numpy.float64'>, 'Northing': <class 'numpy.float64'>}
{'Name': 'str', 'Easting': <class 'numpy.float64'>, 'Northing': <class 'numpy.float64'>}


,Type,SubType,Name,Easting,Northing
0,RailwayStations,NaN,Penzance Rail Station,147588.0,30599.0
1,RailwayStations,NaN,St Ives (Cornwall) Rail Station,151947.0,40127.0
2,RailwayStations,NaN,Carbis Bay Rail Station,152930.0,38745.0
3,RailwayStations,NaN,St Erth Rail Station,154150.0,35730.0
4,RailwayStations,NaN,Lelant Saltings Rail Station,154430.0,36640.0
...,...,...,...,...,...
3183629,NSPL,NaN,ZE3 9JW,438975.0,1110038.0
3183630,NSPL,NaN,ZE3 9JX,438872.0,1110219.0
3183631,NSPL,NaN,ZE3 9JY,438498.0,1112029.0
3183632,NSPL,NaN,ZE3 9JZ,438662.0,1112122.0
